# TVLA Testing of Cryptographic Function

This tutorial will perform a basic TVLA test. Here we are only using an unprotected software implementation, so there is little hope of passing the test! But this can demonstrate how the TVLA test can be useful for validating your crypto implementation.

## Capturing Power Traces

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [ ]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
#%run "Helper_Scripts/Program_XMEGA.ipynb"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"

In [ ]:
# program the target
program_target(scope, fw_path)

In addition, before we capture our traces, we'll need to create a ChipWhipserer project, since that's what Analyzer expects for an input:

In [ ]:
from chipwhisperer.common.api.ProjectFormat import ProjectFormat
project = ProjectFormat()
project.setFilename("jupyter_tvla_sw.cwp")

And we can get the class used to hold our traces by:

In [ ]:
from datetime import datetime
import copy

tc = copy.copy(project.getTraceFormat())
starttime = datetime.now()
prefix = starttime.strftime('%Y.%m.%d-%H.%M.%S') + "_"
tc.config.setConfigFilename(project.datadirectory + "traces/config_" + prefix + ".cfg")
tc.config.setAttr("prefix", prefix)
tc.config.setAttr("date", starttime.strftime('%Y-%m-%d %H:%M:%S'))

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and our new trace into our trace class. We'll also keep track of our keys manually for checking our answer later.

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.tvlattest import AcqKeyTextPattern_TVLATTest
import numpy as np
import time

N = 50  # Number of traces

ktp = AcqKeyTextPattern_TVLATTest(target=target)
ktp.initPair(N)

keys = []
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here
    
    key, text = ktp.newPair()  # TVLA T-Test changes PT between two options
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    #traces.append(scope.getLastTrace())
    tc.addTrace(scope.getLastTrace(), text, "", key)
    
#Calling closeAll() to ensure flush to disk happens
tc.closeAll()

Now that we have our traces, we need to tell the project that the traces are loaded and add them to the project's trace manager.

In [ ]:
project.traceManager().appendSegment(tc)

#Save project file
project.save()

We're now done with the ChipWhisperer hardware, so we should disconnect from the scope and target:

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

Alright! The following chunk of code does the entire TVLA algorithm itself, based on the recorded data:

In [ ]:
from chipwhisperer.common.api.ProjectFormat import ProjectFormat
from matplotlib.pylab import *
import numpy as np
import scipy
import scipy.stats

project = ProjectFormat()
project.load('./jupyter_tvla_sw.cwp')

tm = project.traceManager()
fixedpy = [0xda, 0x39, 0xa3, 0xee, 0x5e, 0x6b, 0x4b, 0x0d, 0x32, 0x55, 0xbf, 0xef, 0x95, 0x60, 0x18, 0x90]

def do_the_ttvla(tm, ntraces=-1):
    if ntraces == -1:
        ntraces = int(tm.numTraces() / 2)
        
    if ntraces * 2 > tm.numTraces():
        raise ValueError("Invalid ntraces")
        
    for g in range(0, 2):
        group = [(tm.getTextin(i) == fixedpy).all() for i in range(g*ntraces, g*ntraces+ntraces)]
        trace = np.zeros((ntraces, tm.numPoints()))

        for n in range(g*ntraces, g*ntraces+ntraces):
            trace[n - g*ntraces][:] = tm.getTrace(n)
                
        testout = welch_ttest(group, trace)
        plot(testout)
        
    plot([0, tm.numPoints()], [-4.5, -4.5], 'r')
    plot([0, tm.numPoints()], [4.5, 4.5], 'r')

def welch_ttest(group, traces):
    # Compute Welch's t-statistic at each point in time
    # Here, group[] must only contain booleans (True/False)
    traces_true = traces[np.where(np.array(group))]
    traces_false = traces[np.where(~np.array(group))]
    
    if len(traces_true) == 0:
        traces_true  = np.array([[np.nan for _ in range(len(traces[0]))]])
    if len(traces_false) == 0:
        traces_false = np.array([[np.nan for _ in range(len(traces[0]))]])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ttrace = scipy.stats.ttest_ind(traces_true, traces_false, axis=0, equal_var=False)[0]
        
    return np.nan_to_num(ttrace) 
    
do_the_ttvla(tm)
title("TVLA Results")
show()

With multiple excursions beyond the 4.5 limit, you can see that the implementation easily fails! The only thing we haven't done right here is figure out *where* the actual crypto stards and ends. This is best done using a T-Test or similar targetting the input and output data of the crypto operation. In this example we didn't even cover the complete portion of the algorithm, but this is easier to do with hardware crypto.

If you'd like to perform this test, good news, we have provided those traces as part of this training example. To use this, simply change the file pointed to as follows:

```
project.load('./tut_data/stm32f415_tvla_1k.cwp')
```

You should see a more obvious location where the T-Test has failed on this result. Note you need to be careful as some of the spikes are actually the data load and unload operations (at the start and end of the algorithm), where we apply the T-Test only on the middle portion of the algorithm.